# Title
**Author**:  Greg Slater <br>
**Date**:  23rd October 2024 <br>
**Dataset Scope**: `dataset` <br>
**Report Type**: Ad-hoc analysis <br>

## Purpose
Try and identify where there are duplicate entities for the same brownfield-land provisions.


In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import urllib
import shapely.wkt
# from sqlite_query_functions import DatasetSqlite
from datetime import datetime

td = datetime.today().strftime('%Y-%m-%d')

pd.set_option("display.max_rows", 100)

data_dir = "output/"
os.makedirs(data_dir, exist_ok=True)


In [91]:
def get_pdp_dataset(dataset, geometry_field = "geometry", crs_out=27700, underscore_cols=True):

    df = pd.read_csv(f"https://files.planning.data.gov.uk/dataset/{dataset}.csv", dtype = "str")
    df.columns = [x.replace("-", "_") for x in df.columns]

    df_valid_geom = df[df[geometry_field].notnull()].copy()

    # load geometry and create GDF
    df_valid_geom[geometry_field] = df_valid_geom[geometry_field].apply(shapely.wkt.loads)
    gdf = gpd.GeoDataFrame(df_valid_geom, geometry = geometry_field)

    # Transform to ESPG:27700 for more interpretable area units
    gdf.set_crs(epsg=4326, inplace=True)
    gdf.to_crs(epsg=crs_out, inplace=True)

    return gdf

def get_all_organisations():
    params = urllib.parse.urlencode({
        "sql": f"""
        select organisation, name as organisation_name, entity as organisation_entity, statistical_geography
        from organisation
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/digital-land.csv?{params}"
    df = pd.read_csv(url, dtype = str)
    return df

## Data Import

In [92]:
# get bfl and org data
pdp_gdf = get_pdp_dataset("brownfield-land", "point", crs_out=4326)
lookup_org = get_all_organisations()

# make string version of point geom field
pdp_gdf["point_str"] = pdp_gdf["point"].apply(shapely.to_wkt)

pdp_gdf = pdp_gdf.merge(
    lookup_org[["organisation_entity", "organisation_name"]],
    how = "left",
    on = "organisation_entity"
)

print(len(pdp_gdf))
pdp_gdf.head()

33796


,dataset,end_date,entity,entry_date,geojson,geometry,name,organisation_entity,point,prefix,...,ownership_status,planning_permission_date,planning_permission_history,planning_permission_status,planning_permission_type,site,site_address,site_categories,site_plan_url,organisation_name
0,brownfield-land,NaN,1700000,2022-11-23,NaN,None,BFR001,336,POINT (-3.49379 50.54974),brownfield-land,...,not owned by a public authority,2001-10-26,NaN,permissioned,full planning permission,NaN,"Glendaragh, Barn Park Road, Teignmouth TQ14 8PN",NaN,http://www.teignbridge.gov.uk/planning/forms/p...,Teignbridge District Council
1,brownfield-land,NaN,1700001,2022-11-23,NaN,None,BFR002,336,POINT (-3.62626 50.53384),brownfield-land,...,not owned by a public authority,2006-03-22,NaN,permissioned,full planning permission,NaN,"Rosemary Avenue, Newton Abbot TQ12 1SB",NaN,http://www.teignbridge.gov.uk/planning/forms/p...,Teignbridge District Council
2,brownfield-land,2019-07-24,1700002,2022-11-23,NaN,None,BFR003,336,POINT (-3.51067 50.53990),brownfield-land,...,not owned by a public authority,2008-07-01,NaN,permissioned,full planning permission,NaN,"Platway House, Torquay Road, Shaldon TQ14 0AU",NaN,http://www.teignbridge.gov.uk/planning/forms/p...,Teignbridge District Council
3,brownfield-land,2018-03-16,1700003,2022-11-23,NaN,None,BFR004,336,POINT (-3.46679 50.58151),brownfield-land,...,not owned by a public authority,2012-07-31,NaN,permissioned,full planning permission,NaN,"Shapters Yard, Strand Hill, Dawlish EX7 9PS",NaN,http://www.teignbridge.gov.uk/planning/forms/p...,Teignbridge District Council
4,brownfield-land,2020-06-16,1700004,2022-11-23,NaN,None,BFR005,336,POINT (-3.62069 50.52059),brownfield-land,...,not owned by a public authority,2013-01-21,NaN,permissioned,full planning permission,NaN,"Former Wolborough Hospital, Old Totnes Road, N...",NaN,http://www.teignbridge.gov.uk/planning/forms/p...,Teignbridge District Council


## Analysis

In [101]:
fields = ["entity", "reference", "name", "organisation_entity", "organisation_name", "point"]

# spatial join on point
overlay = gpd.overlay(
    pdp_gdf[fields],
    pdp_gdf[fields],
    how = "intersection"
)

# add in point string field
overlay["point_str"] = overlay["geometry"].apply(shapely.to_wkt)

print(len(pdp_gdf))
print(len(overlay))
overlay.head()

33796
44818


,entity_1,reference_1,name_1,organisation_entity_1,organisation_name_1,entity_2,reference_2,name_2,organisation_entity_2,organisation_name_2,geometry,point_str
0,1700000,BFR001,BFR001,336,Teignbridge District Council,1700000,BFR001,BFR001,336,Teignbridge District Council,POINT (-3.49379 50.54974),POINT (-3.493786 50.549744)
1,1700001,BFR002,BFR002,336,Teignbridge District Council,1700001,BFR002,BFR002,336,Teignbridge District Council,POINT (-3.62626 50.53384),POINT (-3.626255 50.533836)
2,1700002,BFR003,BFR003,336,Teignbridge District Council,1700002,BFR003,BFR003,336,Teignbridge District Council,POINT (-3.51067 50.53990),POINT (-3.510666 50.5399)
3,1700003,BFR004,BFR004,336,Teignbridge District Council,1700003,BFR004,BFR004,336,Teignbridge District Council,POINT (-3.46679 50.58151),POINT (-3.466788 50.58151)
4,1700004,BFR005,BFR005,336,Teignbridge District Council,1700004,BFR005,BFR005,336,Teignbridge District Council,POINT (-3.62069 50.52059),POINT (-3.620686 50.520592)


In [102]:
# remove self-intersections
overlay_unique = overlay[overlay["entity_1"] != overlay["entity_2"]].copy()

# keep only first version of e1-e2 match records (i.e. keep e1-e2 and bin e2-e1)
overlay_unique["entity_join"] = overlay_unique.apply(lambda x: '-'.join(sorted(x[["entity_1", "entity_2"]])), axis=1)
overlay_unique.sort_values(["entity_join"], ascending=True, inplace=True)
overlay_unique.drop_duplicates(subset="entity_join", inplace = True)  #Drop them by name

print(len(overlay_unique))
# overlay_unique.head()

5511


,entity_1,reference_1,name_1,organisation_entity_1,organisation_name_1,entity_2,reference_2,name_2,organisation_entity_2,organisation_name_2,geometry,point_str,entity_join
18,1700036,BFR037,BFR037,336,Teignbridge District Council,1700017,BFR018,BFR018,336,Teignbridge District Council,POINT (-3.62367 50.51579),POINT (-3.623669 50.515793),1700017-1700036
53,1700056,SB0387,SB0387,280,South Bucks District Council,1700050,SB0034,SB0034,280,South Bucks District Council,POINT (-0.66000 51.53000),POINT (-0.66 51.53),1700050-1700056
54,1727285,SB0399,SB0399,280,South Bucks District Council,1700050,SB0034,SB0034,280,South Bucks District Council,POINT (-0.66000 51.53000),POINT (-0.66 51.53),1700050-1727285
67,1700055,SB0339,SB0339,280,South Bucks District Council,1700066,SB0452,SB0452,280,South Bucks District Council,POINT (-0.66000 51.52000),POINT (-0.66 51.52),1700055-1700066
57,1727285,SB0399,SB0399,280,South Bucks District Council,1700056,SB0387,SB0387,280,South Bucks District Council,POINT (-0.66000 51.53000),POINT (-0.66 51.53),1700056-1727285


### Issue type #2: Cross-organisation fact mix ups
something is going wrong here, looks like fact mix up - to get further investigation

In [163]:
# Cross-org geometry duplicates 
overlay_unique[
    (overlay_unique["organisation_entity_1"] != overlay_unique["organisation_entity_2"]) &
    (overlay_unique["reference_1"] == overlay_unique["reference_2"])
]

,entity_1,reference_1,name_1,organisation_entity_1,organisation_name_1,entity_2,reference_2,name_2,organisation_entity_2,organisation_name_2,geometry,point_str,entity_join
1025,1707735,BR193,BR193,92,Cornwall Council,1700786,BR193,BR193,380,Wokingham Borough Council,POINT (-0.89158 51.38603),POINT (-0.891575 51.386028),1700786-1707735
2346,1736755,BLR2019001,BLR2019001,380,Wokingham Borough Council,1701849,BLR2019001,BLR2019001,379,Woking Borough Council,POINT (-0.56600 51.30500),POINT (-0.566 51.305),1701849-1736755
2351,1736822,BLR2019002,BLR2019002,380,Wokingham Borough Council,1701850,BLR2019002,BLR2019002,379,Woking Borough Council,POINT (-0.55400 51.32200),POINT (-0.554 51.322),1701850-1736822
2366,1701851,BLR2019003,BLR2019003,379,Woking Borough Council,1736826,BLR2019003,BLR2019003,380,Wokingham Borough Council,POINT (-0.55700 51.31700),POINT (-0.557 51.317),1701851-1736826
2375,1736756,BLR2019004,BLR2019004,380,Wokingham Borough Council,1701852,BLR2019004,BLR2019004,379,Woking Borough Council,POINT (-0.55700 51.31800),POINT (-0.557 51.318),1701852-1736756
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39002,1729763,BR027,BR027,234,North Lincolnshire Council,1735109,BR027,BR027,55,Blackpool Borough Council,POINT (-3.01696 53.78519),POINT (-3.01696 53.785194),1729763-1735109
39006,1729764,BR028,BR028,395,City of York Council,1735110,BR028,BR028,55,Blackpool Borough Council,POINT (-3.01294 53.80206),POINT (-3.012943 53.80206),1729764-1735110
42690,1734662,BF052,BF052,321,Stratford-on-Avon District Council,1735148,BF052,BF052,243,Nuneaton and Bedworth Borough Council,POINT (-1.52870 52.53000),POINT (-1.5287 52.53),1734662-1735148
42694,1734663,BF053,BF053,26,Adur District Council,1735149,BF053,BF053,243,Nuneaton and Bedworth Borough Council,POINT (-1.48320 52.52000),POINT (-1.4832 52.52),1734663-1735149


### Issue type #1: Within-org geo duplicates
These are entities which have the same geometry, the same organisation, and different reference values. In many cases these look to be separate records which share an address (and therefore point geometry). e.g. Croydon entities `1726478` and `1726479`, which have the same POINT geom but references of `17/05535/GPDO` and `17/05537/GPDO`. However, there are some which have a different reference value pattern so one may be outdated. e.g. Croydon entity `1734312` which has a reference of `17/05535/GP`. These can all be reliably identified through the new deleted entities report, which shows that this reference value doesn’t exist in the latest resource so should be retired or end-dated.   

Note - the extremely high counts for Merton in the table below are because of many-many joins between empty point geometry - this makes this issue look a lot worse than it is. There are actually just 72 Merton entities with empty point geometry.

No further action, this should be completed as part of ticket when we have an entity end-date resolution with Data Design.

In [115]:
# count number of dupe issues per org
overlay_unique[
    (overlay_unique["organisation_entity_1"] == overlay_unique["organisation_entity_2"]) &
    (overlay_unique["reference_1"] != overlay_unique["reference_2"])
].groupby(["organisation_entity_1", "organisation_name_1"], as_index = False).size().sort_values(by = "size", ascending = False)

,organisation_entity_1,organisation_name_1,size
54,217,London Borough of Merton,2561
1,100,London Borough of Croydon,236
154,63,Brentwood Borough Council,148
49,207,Manchester City Council,145
14,131,East Riding of Yorkshire Council,116
...,...,...,...
84,299,Stockport Metropolitan Borough Council,1
83,297,Shropshire Council,1
79,286,Sedgemoor District Council,1
72,270,Rossendale Borough Council,1


In [166]:
# check records for an org
# overlay_unique[
#     (overlay_unique["organisation_entity_1"] == "217") 
#                ]

In [150]:
# counting the number of duped entities per point for each issue
point_ent_dupe_count = pdp_gdf[pdp_gdf["point_str"].isin(overlay_unique["point_str"])].groupby(["point_str", "organisation_entity", "organisation_name"], as_index = False).agg(
    n_entities = ("entity", "count")
).sort_values("n_entities", ascending = False)

# point_ent_dupe_count[point_ent_dupe_count["n_entities"] == 1]
point_ent_dupe_count

,point_str,organisation_entity,organisation_name,n_entities
2000,POINT (0 0),217,London Borough of Merton,72
2079,POINT (0.244798 51.650988),63,Brentwood Borough Council,17
1829,POINT (-2.46 50.61),112,Dorset Council,10
1998,POINT (-7.55716 49.766809),242,North Tyneside Council,8
227,POINT (-0.115445 51.395333),100,London Borough of Croydon,8
...,...,...,...,...
1339,POINT (-1.820091 50.730345),54,"Bournemouth, Christchurch and Poole Council",1
787,POINT (-0.728796 51.6236),393,Wycombe District Council,1
1975,POINT (-3.01696 53.785194),55,Blackpool Borough Council,1
1974,POINT (-3.01696 53.785194),234,North Lincolnshire Council,1


In [176]:
# look at specific org 
point_ent_dupe_count[point_ent_dupe_count["organisation_entity"] == "112"]

,point_str,organisation_entity,organisation_name,n_entities
1829,POINT (-2.46 50.61),112,Dorset Council,10
1823,POINT (-2.45 50.61),112,Dorset Council,3
1822,POINT (-2.45 50.6),112,Dorset Council,3
1805,POINT (-2.3 50.93),112,Dorset Council,3
1818,POINT (-2.42 50.71),112,Dorset Council,3
1922,POINT (-2.76 50.73),112,Dorset Council,3
1543,POINT (-1.97 50.8),112,Dorset Council,2
1536,POINT (-1.94 50.8),112,Dorset Council,2
1502,POINT (-1.91 50.79),112,Dorset Council,2
1550,POINT (-1.99 50.8),112,Dorset Council,2


In [177]:
# look at platform data for specific point
pdp_gdf[pdp_gdf["point_str"] == "POINT (-2.46 50.61)"]

,dataset,end_date,entity,entry_date,geojson,geometry,name,organisation_entity,point,prefix,...,planning_permission_date,planning_permission_history,planning_permission_status,planning_permission_type,site,site_address,site_categories,site_plan_url,organisation_name,point_str
6081,brownfield-land,NaN,1706506,2020-01-04,NaN,None,LA/WEYM/029,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,Westwey Road (South),NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6117,brownfield-land,NaN,1706542,2020-01-04,NaN,None,LA/WEYM/030,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,"Bus Depot, King Street",NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6118,brownfield-land,NaN,1706543,2020-01-04,NaN,None,LA/WEYM/031,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,"Harbourside Car Park, Commercial Road",NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6120,brownfield-land,NaN,1706545,2020-01-04,NaN,None,LA/WEYM/033,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,"Lakeside Superbowl, St Nicholas Street",NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6121,brownfield-land,NaN,1706546,2020-01-04,NaN,None,LA/WEYM/034,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,Land at Commercial Road (car park),NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6123,brownfield-land,NaN,1706548,2020-01-04,NaN,None,LA/WEYM/036,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,Post Office Sorting Office,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6124,brownfield-land,NaN,1706549,2020-01-04,NaN,None,LA/WEYM/037,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,"The Loop Car Park, Commercial Road",NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6125,brownfield-land,NaN,1706550,2020-01-04,NaN,None,LA/WEYM/038,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,"Weymouth Bowling Club, Commercial Road",NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6127,brownfield-land,NaN,1706552,2020-01-04,NaN,None,LA/WEYM/040,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,Land at Park Street (Car Park),NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6128,brownfield-land,NaN,1706553,2020-01-04,NaN,None,LA/WEYM/041,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,"Council Offices, North Quay",NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)


### Issue type #3: source data issues we can fix

Merton empty geom issues seem to be coming from emtpy `GeoX` and `GeoY` fields. Attempted to fix this with PR: https://github.com/digital-land/config/pull/244

In [175]:
# look at platform data for specific point
pdp_gdf[
    (pdp_gdf["point_str"] == "POINT (0 0)") &
    (pdp_gdf["organisation_entity"] == "217")].head()

,dataset,end_date,entity,entry_date,geojson,geometry,name,organisation_entity,point,prefix,...,planning_permission_date,planning_permission_history,planning_permission_status,planning_permission_type,site,site_address,site_categories,site_plan_url,organisation_name,point_str
12446,brownfield-land,NaN,1713536,2020-01-10,NaN,None,96P0219,217,POINT (0.00000 0.00000),brownfield-land,...,1998-08-05,NaN,permissioned,full planning permission,NaN,153-161 The Broadway SW19,NaN,https://maps.london.gov.uk/brownfield-site-reg...,London Borough of Merton,POINT (0 0)
12447,brownfield-land,NaN,1713537,2020-01-10,NaN,None,99P1419,217,POINT (0.00000 0.00000),brownfield-land,...,2000-12-14,NaN,permissioned,full planning permission,NaN,Land At Batsworth Road CR4 3BX,NaN,https://maps.london.gov.uk/brownfield-site-reg...,London Borough of Merton,POINT (0 0)
12448,brownfield-land,NaN,1713538,2020-01-10,NaN,None,01/P0805,217,POINT (0.00000 0.00000),brownfield-land,...,2004-06-10,NaN,permissioned,NaN,NaN,122-136 London Road CR4 3LB,NaN,https://maps.london.gov.uk/brownfield-site-reg...,London Borough of Merton,POINT (0 0)
12449,brownfield-land,NaN,1713539,2020-01-10,NaN,None,01/P1596,217,POINT (0.00000 0.00000),brownfield-land,...,2001-11-15,NaN,permissioned,full planning permission,NaN,32 - 34 South Lodge Avenue CR4 1LU,NaN,https://maps.london.gov.uk/brownfield-site-reg...,London Borough of Merton,POINT (0 0)
12450,brownfield-land,NaN,1713540,2020-01-10,NaN,None,03/P0216,217,POINT (0.00000 0.00000),brownfield-land,...,2003-11-12,NaN,permissioned,full planning permission,NaN,Land Adjacent To 4 Caesars Walk CR4 4LE,NaN,https://maps.london.gov.uk/brownfield-site-reg...,London Borough of Merton,POINT (0 0)


### Issue type 43: source data issues we cannot fix

Some orgs, like Dorset, look like they do just have bad point data. Taking the example of entities with `POINT (-2.46 50.61)` from the table below, we can see that these entities all have different addresses despite having identical geometries. So probably something that needs to be fixed at source.

In [ ]:
# look at specific org 
point_ent_dupe_count[point_ent_dupe_count["organisation_entity"] == "112"]

,point_str,organisation_entity,organisation_name,n_entities
1829,POINT (-2.46 50.61),112,Dorset Council,10
1823,POINT (-2.45 50.61),112,Dorset Council,3
1822,POINT (-2.45 50.6),112,Dorset Council,3
1805,POINT (-2.3 50.93),112,Dorset Council,3
1818,POINT (-2.42 50.71),112,Dorset Council,3
1922,POINT (-2.76 50.73),112,Dorset Council,3
1543,POINT (-1.97 50.8),112,Dorset Council,2
1536,POINT (-1.94 50.8),112,Dorset Council,2
1502,POINT (-1.91 50.79),112,Dorset Council,2
1550,POINT (-1.99 50.8),112,Dorset Council,2


In [ ]:
# look at platform data for specific point
pdp_gdf[pdp_gdf["point_str"] == "POINT (-2.46 50.61)"]

,dataset,end_date,entity,entry_date,geojson,geometry,name,organisation_entity,point,prefix,...,planning_permission_date,planning_permission_history,planning_permission_status,planning_permission_type,site,site_address,site_categories,site_plan_url,organisation_name,point_str
6081,brownfield-land,NaN,1706506,2020-01-04,NaN,None,LA/WEYM/029,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,Westwey Road (South),NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6117,brownfield-land,NaN,1706542,2020-01-04,NaN,None,LA/WEYM/030,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,"Bus Depot, King Street",NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6118,brownfield-land,NaN,1706543,2020-01-04,NaN,None,LA/WEYM/031,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,"Harbourside Car Park, Commercial Road",NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6120,brownfield-land,NaN,1706545,2020-01-04,NaN,None,LA/WEYM/033,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,"Lakeside Superbowl, St Nicholas Street",NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6121,brownfield-land,NaN,1706546,2020-01-04,NaN,None,LA/WEYM/034,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,Land at Commercial Road (car park),NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6123,brownfield-land,NaN,1706548,2020-01-04,NaN,None,LA/WEYM/036,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,Post Office Sorting Office,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6124,brownfield-land,NaN,1706549,2020-01-04,NaN,None,LA/WEYM/037,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,"The Loop Car Park, Commercial Road",NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6125,brownfield-land,NaN,1706550,2020-01-04,NaN,None,LA/WEYM/038,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,"Weymouth Bowling Club, Commercial Road",NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6127,brownfield-land,NaN,1706552,2020-01-04,NaN,None,LA/WEYM/040,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,Land at Park Street (Car Park),NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
6128,brownfield-land,NaN,1706553,2020-01-04,NaN,None,LA/WEYM/041,112,POINT (-2.46000 50.61000),brownfield-land,...,NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,not permissioned,NaN,NaN,"Council Offices, North Quay",NaN,https://www.dorsetcouncil.gov.uk/planning-buil...,Dorset Council,POINT (-2.46 50.61)
